In [1]:
import os, sys
from pathlib import Path
import cortex
import nibabel as nib
import numpy as np
import matplotlib.colors as colors
import matplotlib.pyplot as pl
import time
import platform

In [2]:
if platform.node()=='theCryingMachine.local':
    deriv_dir = Path('/Users/ningkai/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-001/derivatives/')
elif platform.node()=='Smoldering-Corpse-Bar.local':
    deriv_dir = Path('/Users/dionysus/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-001/derivatives/')
elif platform.node()=='node230':
    raise ValueError('hmmm I dont think it works on a server. Your are working on {}'.format(platform.node()))
else:
    raise ValueError('No idea which computer this is. The computer name is {}'.format(platform.node()))

print('Running on {}'.format(platform.node()))
print('Current deriv folder is {}'.format(deriv_dir))
data_dir = Path(deriv_dir, 'GLMsingle')


Running on theCryingMachine.local
Current deriv folder is /Users/ningkai/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-001/derivatives


In [3]:
print('cortex.database.default_filestore: {}'.format(cortex.database.default_filestore))
print('cortex.options.usercfg: {}'.format(cortex.options.usercfg))

cortex.database.default_filestore: /Users/ningkai/Documents/pycortex/db
cortex.options.usercfg: /Users/ningkai/Library/Application Support/pycortex/options.cfg


In [8]:
# import data into pycortex
subj_nr = 1

sub = "sub-%s" % str(subj_nr).zfill(3)
fs_dir = Path(deriv_dir, 'freesurfer')
fs_subj_dir = Path(fs_dir, 'sub-001')
# pial_path = Path(fs_subj_dir, 'surf', 'lh.pial')
ln_pial_path = Path(fs_subj_dir, 'surf', 'lh.pial')
# os.system("ln -s {} {}".format(pial_path, ln_pial_path))


xfmname = "fmriprep_fsT1"
dd = Path(data_dir, sub, "ses-1")
epi = Path(dd, "reg", "epi.nii.gz")
epi_2_T1 = Path(dd, "reg", "bbr.mat")
T1 = Path(fs_dir, sub, "mri", "T1.nii.gz")
bg_image_fn = Path(deriv_dir, 'func', 'sub-001_ses-01_task-ping_run-5_space-T1w_boldref.nii.gz')
bm_image_fn = Path(deriv_dir, 'func', 'sub-001_ses-01_task-ping_run-5_space-T1w_desc-brain_mask.nii.gz')
bref = nib.load(bg_image_fn)
bmask = nib.load(bm_image_fn).get_fdata().astype(bool)
print('bref.shape: {}'.format(bref.shape))
print('bmask.shape: {}'.format(bmask.shape))

bref.shape: (91, 106, 85)
bmask.shape: (91, 106, 85)


In [12]:
np.identity(4)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [18]:
bref.affine

array([[   1.68799996,    0.        ,    0.        ,  -76.11100006],
       [   0.        ,    1.68799996,    0.        , -101.10510254],
       [   0.        ,    0.        ,    1.70000005,  -56.66110229],
       [   0.        ,    0.        ,    0.        ,    1.        ]])

In [32]:
a = np.identity(4)
transform2 = cortex.xfm.Transform(np.identity(4), bref)
print(transform2.xfm)
print(transform2.shape)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
(85, 106, 91)


In [5]:

os.system("mri_convert %s %s" % (Path(str(T1).replace('.nii.gz', '.mgz')), T1))

cortex.freesurfer.import_subj(fs_subject='sub-'+str(subj_nr).zfill(3),
                                cx_subject='sub-'+str(subj_nr).zfill(3),
                                freesurfer_subject_dir=fs_dir)
# cortex.freesurfer.import_flat(fs_subject='sub-'+str(subj_nr).zfill(3),
#                                 patch='full',
#                                 freesurfer_subject_dir=fs_dir)
# add_session_2_cortex(epi_2_T1, epi, T1, sub, xfmname, transpose=False)

transform = cortex.xfm.Transform(np.identity(4), bref)
transform.save('sub-'+str(subj_nr).zfill(3), xfmname, 'magnet')

mri_convert /Users/ningkai/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-001/derivatives/freesurfer/sub-001/mri/T1.mgz /Users/ningkai/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-001/derivatives/freesurfer/sub-001/mri/T1.nii.gz 
reading from /Users/ningkai/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-001/derivatives/freesurfer/sub-001/mri/T1.mgz...
TR=0.00, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, 0, 0)
j_ras = (0, 2.14097e-08, -1)
k_ras = (0, 1, 2.14097e-08)
writing to /Users/ningkai/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-001/derivatives/freesurfer/sub-001/mri/T1.nii.gz...
b'created by nwa201 on Tue Oct 31 08:15:34 2023\n'
b'created by nwa201 on Tue Oct 31 13:54:12 2023\n'
b'created by nwa201 on Tue Oct 31 11:13:19 2023\n'
b'created by nwa201 on Tue Oct 31 13:55:41 2023\n'
b'created by nwa201 on Tue Oct 31 08:15:34 2023\n'
b'created by nwa201 on Tue Oc

In [35]:
for xfm_mask in ["cortical", "thick", "thin", "nearest"]:
    print('dealing with mask: {}'.format(xfm_mask))
    mask = cortex.get_cortical_mask(subject='sub-'+str(subj_nr).zfill(3), xfmname=xfmname, type=xfm_mask)
    mask_img = nib.Nifti1Image(dataobj=mask.transpose((2,1,0)), affine=bref.affine, header=bref.header)
    print('mask_img.shape: {}'.format(mask_img.shape))
    mask_file = "{}/db/{}/transforms/{}/mask_{}.nii.gz".format(cortex.database.default_filestore, 'sub-'+str(subj_nr).zfill(3), xfmname, xfm_mask)
    mask_img.to_filename(mask_file)

dealing with mask: cortical
0.000%
0.044%
0.089%
0.133%
0.177%
0.221%
0.266%
0.310%
0.354%
0.398%
0.443%
0.487%
0.531%
0.576%
0.620%
0.664%
0.708%
0.753%
0.797%
0.841%
0.886%
0.930%
0.974%
1.018%
1.063%
1.107%
1.151%
1.195%
1.240%
1.284%
1.328%
1.373%
1.417%
1.461%
1.505%
1.550%
1.594%
1.638%
1.683%
1.727%
1.771%
1.815%
1.860%
1.904%
1.948%
1.992%
2.037%
2.081%
2.125%
2.170%
2.214%
2.258%
2.302%
2.347%
2.391%
2.435%
2.479%
2.524%
2.568%
2.612%
2.657%
2.701%
2.745%
2.789%
2.834%
2.878%
2.922%
2.967%
3.011%
3.055%
3.099%
3.144%
3.188%
3.232%
3.276%
3.321%
3.365%
3.409%
3.454%
3.498%
3.542%
3.586%
3.631%
3.675%
3.719%
3.763%
3.808%
3.852%
3.896%
3.941%
3.985%
4.029%
4.073%
4.118%
4.162%
4.206%
4.251%
4.295%
4.339%
4.383%
4.428%
4.472%
4.516%
4.560%
4.605%
4.649%
4.693%
4.738%
4.782%
4.826%
4.870%
4.915%
4.959%
5.003%
5.048%
5.092%
5.136%
5.180%
5.225%
5.269%
5.313%
5.357%
5.402%
5.446%
5.490%
5.535%
5.579%
5.623%
5.667%
5.712%
5.756%
5.800%
5.844%
5.889%
5.933%
5.977%
6.022%
6.066%
6.110%

KeyboardInterrupt: 

In [41]:
results_glmsingle = dict()

results_glmsingle['typea'] = np.load(Path(data_dir,'TYPEA_ONOFF.npy'),allow_pickle=True).item()
results_glmsingle['typeb'] = np.load(Path(data_dir,'TYPEB_FITHRF.npy'),allow_pickle=True).item()
results_glmsingle['typec'] = np.load(Path(data_dir,'TYPEC_FITHRF_GLMDENOISE.npy'),allow_pickle=True).item()
results_glmsingle['typed'] = np.load(Path(data_dir,'TYPED_FITHRF_GLMDENOISE_RR.npy'),allow_pickle=True).item()
R2_typec_masked = np.zeros_like(results_glmsingle['typec']['R2'])
R2_typec_masked[bmask] = results_glmsingle['typec']['R2'][bmask]

R2_typec_masked_cortex = cortex.Volume(R2_typec_masked.transpose((2,1,0)), 'sub-001', xfmname, cmap='BuBkRd')


offset_typed_thres_masked = nib.load(Path(data_dir, 'offset_typed_thres_masked.nii.gz')).get_fdata()
offset_typed_thres_masked_cortex = cortex.Volume(offset_typed_thres_masked.transpose((2,1,0)), 'sub-001', xfmname, cmap='hsv')

In [44]:
cortex.webshow(offset_typed_thres_masked_cortex)

Started server on port 36150
Stopping server


<JS: window.viewer>

In [43]:
cortex.webgl.make_static(outpath=deriv_dir, data=offset_typed_thres_masked_cortex)